# Sharing Variables

In [1]:
import tensorflow as tf

## 问题

 想象一下你要为图像过滤任务创建一个简单的CNN模型，参数还是用`tf.Variable`表示，代码如下：

In [2]:
def my_image_filter(input_images):
    conv1_weights = tf.Variable(tf.random_normal([5, 5, 32, 32]), name="conv1_weights")
    conv1_biases = tf.Variable(tf.zeros([32]), name="conv1_biases")
    conv1 = tf.nn.conv2d(input_images, conv1_weigths,
                        strides=[1, 1, 1, 1], padding='SAME')
    
    relu1 = tf.nn.relu(conv1 + conv1_biases)
    
    conv2_weights = tf.Variable(tf.random_normal([5, 5, 32, 32]), name="conv2_weights")
    conv2_biases = tf.Variable(tf.zeros([32]), name="conv2_biases")
    conv2 = tf.nn.conv2d(relu1, conv2_weights,
                        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2 + conv2_biases)

虽然模型只有两个卷积层，但是却有4个不同的参数变量：`conv1_weigths`, `conv1_biases`, `conv2_weights`, `conv2_biases`。

特别地，如果你想要重复使用这段代码，比如用于两个不同的图像数据集image1和image2。你为每个数据集调用此方法，然而，tensorflow却创建了两组参数变量！，也就是创建了8=2*4个变量。

In [ ]:
# 第一次调用会创建4个变量
result1 = my_image_filter(image1)
# 第二次调用会创建另外4个变量
result2 = my_image_filter(image2)

能否共享变量呢？即，只创建一组（4个）变量，每个数据集都共享这组变量？

常见的方法是用**全局变量**表示参数：

In [ ]:
variable_dict = {
    "conv1_weights": tf.Variable(tf.random_normal([5, 5, 32, 32]),
                                name = "conv1_weights")
    "conv1_biases": tf.Variable(tf.zeros([32]), name="conv1_biases")
    "conv2_weights": tf.Variable(tf.random_normal([5, 5, 32, 32]),
                                name = "conv2_weights")
    "conv2_biases": tf.Variable(tf.zeros([32]), name="conv2_biases")
    
}

def my_image_filter(input_images, variables_dict):
    conv1 = tf.nn.conv2d(input_images, variables_dict["conv1_weights"],
                        strides=[1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + variables_dict["conv1_biases"])
    
    conv2 = tf.nn.conv2d(relu1, variables_dict["conv2_weights"],
                        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2 + variables_dict["conv2_biases"])

# 调用
result1 = my_image_filter(image1, variables_dict)
result2 = my_image_filter(image2, variables_dict)
    

用全局变量虽然能解决参数共享，但是却破坏了代码的封装性。

TensorFlow提供了**Variable Scope**机制来轻松解决变量共享的问题。

### Variable Scope

TensorFlow中的Variable Scope机制主要包含两个函数：
* tf.get_variable(<name>, <shape>, <initializer>): 创建或返回一个指定name的variable
* tf.variable_scope(<scope_name>): 管理name的命名空间

函数`tf.get_variable()`用于获取或创建一个variable，而不是直接调用tf.Variable。它使用初始化器而不是直接传递一个tensor初始化。

什么是初始化器(initializer)？初始化器实际上是函数，它接收shape作参数，然后返回一个shape相同的tensor。TensorFlow中常用的初始化器包括：
* tf.constant_initializer(value) 用value值进行初始化
* tf.random_uniform_initializer(a, b) 均匀分布[a, b]
* tf.random_normal_initializer(mean, stddev)

我们重构上面的代码：

In [1]:
def conv_relu(input, kernel_shape, bias_shape):
    # 创建名为weights的variable
    weights = tf.get_variable("weights", kernel_shape,
                             initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", bias_shape,
                            initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights, strides=[1,1,1,1], padding='SAME')
    return tf.nn.relu(conv + biases)

函数内部创建了weights和biases两个变量，我们想把这个两个变量用于conv1和conv2，但是参数的名字不应该一样，好了，下面该tf.variable_scope()登场了。

In [2]:
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # 这里创建的变量名字是 conv1/weights, conv1/biases
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # 这里创建的变量名字是 conv2/weights, conv2/biases
        return conv_relu(relu1, [5, 5, 32, 32], [32])

In [ ]:
result1 = my_image_filter(image1)
result2 = my_image_filter(image2)
# Raises ValueError(... conv1/weights already exists...)

正如你看到的，tf.get_variable()会检查已经存在的变量并没有被共享，如果你要共享它们，需要手动指定:

In [ ]:
with tf.variable_scope("image_filters") as scope:
    result1 = my_image_filter(image1)
    scope.reuse_variables()
    result2 = my_image_filter(image2)

### Variable Scope原理

#### 理解 tf.get_variable()： 有同名就报错 vs 找不到就报错

想要理解variable scope，先要知道tf.get_variable()的工作原理，

v = tf.get_variable(name, shape, dtype, initializer)


根据scope的情况，tf.get_variable要做的事情也不同
* Case 1: scope用于创建新变量，也就是 `tf.get_variable_scope().reuse == False`

v是一个新建的tf.Variable，这个v的名字就是scope+参数name，**注意**：tf会检查有没有名字相同的变量已经存在，如果已存在，get_variable会返回`ValueError`（因为reuse==False，有同名的就报错）。

In [3]:
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
assert v.name == "foo/v:0"

* Case 2: scope用于重用变量，也就是 tf.get_variable_scope().reuse == True

此时，get_variable()会根据提供的变量名搜索已经存在的变量，如果不存在指定名字的变量，会报错：ValueError(因为reuse==True，找不到就报错)。如果找到了同名的变量，会把它返回。

In [2]:
with tf.variable_scope("foo"):# 有同名的就报错
    v = tf.get_variable("v", [1])
with tf.variable_scope("foo", reuse=True): #找不到就报错
    v1 = tf.get_variable("v", [1])

assert v1 is v

### tf.variable_scope()基础

知道了`tf.get_variable()`的工作原理也就很容易理解variable scope的含义。variable scope最基本的功能是为变量名提供前缀标示和reuse-flag（即是否重用variable）。variable scope可以嵌套：

In [4]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
assert v.name == "foo/bar/v:0"

调用`tf.get_variable_scope()`可以获取当前的variable scope；

调用 `tf.get_variable_scope().reuse_variables()`可以把当前variable scope的reuse设置为True:

In [6]:
with tf.variable_scope("fooo"):
    v = tf.get_variable("v", [1])
    tf.get_variable_scope().reuse_variables()
    v1 = tf.get_variable("v", [1])
assert v1 is v

**注意你不能再把reuse设置为False**。

variable scope是可以嵌套的，如果某个scope的reuse是True，那么这个scope的子scope的reuse也是True。

In [10]:
with tf.variable_scope("root"):
    # 一开始scope的reuse=False
    assert tf.get_variable_scope().reuse == False
    with tf.variable_scope("foo"):
        # Open a sub-scope, reuse==False
        assert tf.get_variable_scope().reuse == False
    with tf.variable_scope("foo", reuse=True):
        # 显式创建一个reusing scope
        assert tf.get_variable_scope().reuse == True
        with tf.variable_scope("bar"):
            # 子-scope继承了 reuse=True
            assert tf.get_variable_scope().reuse == True
    # root scope还是reuse=False
    assert tf.get_variable_scope().reuse == False

### Capturing variable scope

看一下scope的其他用法, 作参数：

In [11]:
with tf.variable_scope("ffoo") as ffoo_scope:
    v = tf.get_variable("v", [1])
with tf.variable_scope(ffoo_scope):
    w = tf.get_variable("w", [1])
with tf.variable_scope(ffoo_scope, reuse=True):
    v1 = tf.get_variable("v", [1])
    w1 = tf.get_variable("w", [1])

assert v1 is v
assert w1 is w

把variable scope作为参数传递时，新建的variable scope的name就是传入的scope.name，不管新建的scope是否是某个子-scope，它是独立的！

In [12]:
with tf.variable_scope("fooo") as fooo_scope:
    assert fooo_scope.name == "fooo"
with tf.variable_scope("barr"):
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "barr/baz" ########### barr/baz
        with tf.variable_scope(fooo_scope) as fooo_scope2:
            assert fooo_scope2.name == "fooo" ############### 这里不是barr/baz/fooo
        

### variable scope中的初始化器

`tf.get_variable`有一个initializer参数，在创建一个variable时可以设置初始化器，如果我们想为variable scope下的所有variable都设置同样的初始化器，能否不用每次都在调用get_variable时设定初始化器呢？

`tf.variable_scope`也有一个initializer参数，为scope内的所有variable设定初始化器。除非你为某个子scope重新设定初始化器，否则子-scope也会继承这个默认初始化器！

In [ ]:
with tf.variable_scope("ffooo", initializer=tf.constant_initializer(0.5)):
    v = tf.get_variable("v", [1])
    assert v.eval() == 0.5
    w = tf.get_variable("w", [1], initializer=tf.constant_initializer(0.3)):
    assert w.eval() == 0.3
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
        assert v.eval() == 0.5 # 继承了默认初始化器
    with tf.variable_scope("baz", inititlizer=tf.constant_initializer(0.2)):
        v = tf.get_variable("v", [1])
        assert v.eval() == 0.2 # 显式修改默认初始化器

###  tf.variable_scope()内的其他ops的名字

variable scope下的其他ops的名字也会继承scope的前缀。

In [13]:
with tf.variable_scope("fffoo"):
    x = 1.0 + tf.get_variable("v", [1])
assert x.op.name == "fffoo/add"

除了`tf.variable_scope`还有一个`tf.name_scope`， name scope只影响ops的名字，不会影响variable的名字。

In [16]:
with tf.variable_scope("foo3"):
    with tf.name_scope("bar"):
        v = tf.get_variable("v", [1])
        x = 1.0 + v
assert v.name == "foo3/v:0"
assert x.op.name == "foo3/bar/add"